# Azure AI Agent service - Single agent

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [ ]:
#%pip install --upgrade azure-ai-projects

In [5]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.13.4 (main, Jun  3 2025, 15:34:24) [Clang 17.0.0 (clang-1700.0.13.3)]'

## define project client

In [6]:

endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


## Single agent

In [9]:
agent = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are helpful agent",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_Irj2PyhOsiaT6TxcmQefrVpA


In [10]:
agent

{'id': 'asst_Irj2PyhOsiaT6TxcmQefrVpA', 'object': 'assistant', 'created_at': 1749322839, 'name': 'my-agent', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are helpful agent', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}

In [11]:
prompt = "Hello"

In [21]:
    # [START create_thread]
    thread = project_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")



Created thread, thread ID: thread_13imyrGVwad1ydJuH3AGv995


In [22]:
    # [START create_message]
    message = project_client.messages.create(thread_id=thread.id, role="user", content="Hello, tell me a joke")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_f6BzYD7rBI1ygLjmU4R97hvR


In [23]:
thread

{'id': 'thread_13imyrGVwad1ydJuH3AGv995', 'object': 'thread', 'created_at': 1749326948, 'metadata': {}, 'tool_resources': {}}

In [24]:
message

{'id': 'msg_f6BzYD7rBI1ygLjmU4R97hvR', 'object': 'thread.message', 'created_at': 1749326950, 'assistant_id': None, 'thread_id': 'thread_13imyrGVwad1ydJuH3AGv995', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, tell me a joke', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [27]:
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")

Created run, run ID: run_acEjmuJM4KO4shkbOPNxHsBa


In [29]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

Run status: RunStatus.COMPLETED


In [30]:
run

{'id': 'run_acEjmuJM4KO4shkbOPNxHsBa', 'object': 'thread.run', 'created_at': 1749327013, 'assistant_id': 'asst_Irj2PyhOsiaT6TxcmQefrVpA', 'thread_id': 'thread_13imyrGVwad1ydJuH3AGv995', 'status': 'completed', 'started_at': 1749327013, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1749327014, 'required_action': None, 'last_error': None, 'model': 'gpt-4.1', 'instructions': 'You are helpful agent', 'tools': [], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 29, 'completion_tokens': 25, 'total_tokens': 54, 'prompt_token_details': {'cached_tokens': 0}}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}

In [31]:
run.usage

{'prompt_tokens': 29, 'completion_tokens': 25, 'total_tokens': 54, 'prompt_token_details': {'cached_tokens': 0}}

In [32]:
start = run.started_at
end = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  " f"({elapsed.total_seconds():.3f} seconds)")

Start   : 2025-06-07 20:10:13+0000
End     : 2025-06-07 20:10:14+0000
Elapsed : 0:00:01  (1.000 seconds)


In [35]:
    # [START list_messages]
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")
    # [END list_messages]

MessageRole.USER: Hello, tell me a joke
MessageRole.AGENT: Sure! Why did the scarecrow win an award?

Because he was outstanding in his field! 🌾😄


## Another agent

In [45]:
# Create a new agent with different instructions

agent = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are a GenAI agent. Always answer in French",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_PJr1M2mw04VOtgoLxU5fMIVF


In [46]:
    # [START create_thread]
    thread = project_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_0Do5Byspl93VHrs1n5gC9iCj


In [47]:
    # [START create_message]
    message = project_client.messages.create(thread_id=thread.id, role="user", content="What is a GenAI model?")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_C6PkKZTFkPzWbhctZbFIW8Ew


In [48]:
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")

Created run, run ID: run_mCctcJM0TgiYKWiu9FoKSKms


In [49]:
    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")

Run status: RunStatus.COMPLETED


In [50]:
run.usage

{'prompt_tokens': 37, 'completion_tokens': 182, 'total_tokens': 219, 'prompt_token_details': {'cached_tokens': 0}}

In [51]:
start = run.started_at
end = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  " f"({elapsed.total_seconds():.3f} seconds)")

Start   : 2025-06-07 20:17:26+0000
End     : 2025-06-07 20:17:28+0000
Elapsed : 0:00:02  (2.000 seconds)


In [52]:
    # [START list_messages]
    messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")
    # [END list_messages]

MessageRole.USER: What is a GenAI model?
MessageRole.AGENT: Un modèle GenAI, ou modèle d’intelligence artificielle générative, est une intelligence artificielle conçue pour créer de nouveaux contenus, comme du texte, des images, de la musique, du code ou d’autres données, à partir d’exemples existants. Ces modèles sont entraînés sur de grandes quantités de données et utilisent des techniques avancées d’apprentissage automatique, telles que les réseaux de neurones profonds.

Par exemple, les modèles de type GPT (comme celui qui alimente cette conversation) sont capables de générer des réponses, des articles, des histoires ou même du code informatique, en analysant et en s’inspirant des schémas trouvés dans les données sur lesquelles ils ont été formés. Les modèles génératifs sont très utilisés dans de nombreux domaines, notamment l’assistance virtuelle, la création artistique, le service client et bien d’autres applications.


## Cleaning

In [58]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")


In [ ]:

agents = project_client.list_agents()
for agent in agents:
    project_client.delete_agent(agent.id)
print("All agents deleted.")

All agents deleted.


In [60]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")